In [17]:
import numpy as np
from tensorflow.keras.layers import Dense, SimpleRNN, Input, BatchNormalization, LSTM, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

In [2]:
with open('data.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')
    texts = texts.replace('\xa0', '')

In [3]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»„“', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

dist = list(tokenizer.word_counts.items())
print(dist[:20])

[('per', 9), ('dieną', 5), ('žemę', 2), ('lietų', 1), ('godžiai', 1), ('plempia', 1), ('kuriam', 2), ('kas', 5), ('valandą', 1), ('mažiau', 4), ('erdvės', 1), ('stotis', 1), ('bažnyčia', 1), ('kapinaitės', 1), ('plentas', 1), ('ir', 99), ('autobusas', 1), ('kur', 8), ('tuoj', 7), ('pat', 10)]


In [4]:
data = tokenizer.texts_to_sequences([texts])
sak = to_categorical(data[0], num_classes=maxWordsCount)

inp_words = 2
n = sak.shape[0] - inp_words

X = np.array([sak[i:i + inp_words, :] for i in range(n)])
print(X)
Y = sak[inp_words:]
print(Y)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
model = Sequential()
model.add(Input(shape=(inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               144512    
                                                                 
 dense (Dense)               (None, 1000)              129000    
                                                                 
Total params: 273,512
Trainable params: 273,512
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
75/75 [==============================] - 3s 3ms/step - loss: 6.8876 - accuracy: 0.0264
Epoch 2/50
75/75 [==============================] - 0s 3ms/step - loss: 6.4456 - accuracy: 0.0415
Epoch 3/50
75/75 [==============================] - 0s 3ms/step - loss: 6.1732 - accuracy: 0.0415
Epoch 4/50
75/75 [==============================] - 0s 4ms/step - loss: 5.9780 - accuracy: 0.0566
Epoch 5/50
75/75 [==============================] - 0s 3ms/step - loss: 5.7482 - accuracy: 0.1253
Epoch 6/50
75/75 [==============================] - 0s 3ms/step - loss: 5.4809 - accuracy: 0.1597
Epoch 7/50
75/75 [==============================] - 0s 3ms/step - loss: 5.1411 - accuracy: 0.1966
Epoch 8/50
75/75 [==============================] - 0s 4ms/step - loss: 4.7263 - accuracy: 0.2766
Epoch 9/50
75/75 [==============================] - 0s 3ms/step - loss: 4.2383 - accuracy: 0.3554
Epoch 10/50
75/75 [==============================] - 0s 3ms/step - loss: 3.7164 - accuracy: 0.4120
Epoch 11/50
75/75 [

In [7]:
def buildPhrase(word1, word2, str_len=10):
    sak = word1 + " " + word2
    data1 = tokenizer.texts_to_sequences([word1])[0]
    data2 = tokenizer.texts_to_sequences([word2])[0]
    data = [data1[0], data2[0]]
    for i in range(str_len):
        x = to_categorical(data[i: i + inp_words], num_classes=maxWordsCount) # pakeičiame su One-Hot-encoding
        inp = x.reshape(1, inp_words, maxWordsCount)
        pred = model.predict(inp) #prognozavimas
        indx = pred.argmax(axis=1)[0]
        data.append(indx)
        sak += " " + tokenizer.index_word[indx] # papildome sakinį žodžiais
    return sak

In [8]:
sak = buildPhrase("taip", "daro")
print(sak)

1/1 [==============================] - 0s 14ms/step
taip daro žmogus kol jam visur tave ir tegul tik tavęs jam


In [24]:
with open('data.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')
    texts = texts.replace('\xa0', '')


maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»„“', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])
dist = list(tokenizer.word_counts.items())


data = tokenizer.texts_to_sequences([texts])
sak = to_categorical(data[0], num_classes=maxWordsCount)
inp_words = 2
n = sak.shape[0] - inp_words
X = np.array([sak[i:i + inp_words, :] for i in range(n)])
Y = sak[inp_words:]


model = Sequential()
model.add(Input(shape=(inp_words, maxWordsCount)))
model.add(SimpleRNN(128, return_sequences=True))
model.add(SimpleRNN(64, return_sequences=True))
model.add(SimpleRNN(64, return_sequences=True))
model.add(SimpleRNN(64, return_sequences=True))
model.add(SimpleRNN(32,activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=100)

sak = buildPhrase("taip", "daro")
print(sak)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_13 (SimpleRNN)   (None, 2, 128)            144512    
                                                                 
 simple_rnn_14 (SimpleRNN)   (None, 2, 64)             12352     
                                                                 
 simple_rnn_15 (SimpleRNN)   (None, 2, 64)             8256      
                                                                 
 simple_rnn_16 (SimpleRNN)   (None, 2, 64)             8256      
                                                                 
 simple_rnn_17 (SimpleRNN)   (None, 32)                3104      
                                                                 
 dense_7 (Dense)             (None, 1000)              33000     
                                                                 
Total params: 209,480
Trainable params: 209,480
Non-t

In [30]:
with open('data.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')
    texts = texts.replace('\xa0', '')


maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»„“', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])
dist = list(tokenizer.word_counts.items())


data = tokenizer.texts_to_sequences([texts])
sak = to_categorical(data[0], num_classes=maxWordsCount)
inp_words = 2
n = sak.shape[0] - inp_words
X = np.array([sak[i:i + inp_words, :] for i in range(n)])
Y = sak[inp_words:]


model = Sequential()
model.add(Input(shape=(inp_words, maxWordsCount)))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=50)

sak = buildPhrase("taip", "daro")
print(sak)

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 2, 64)             272640    
                                                                 
 lstm_8 (LSTM)               (None, 32)                12416     
                                                                 
 dense_13 (Dense)            (None, 1000)              33000     
                                                                 
Total params: 318,056
Trainable params: 318,056
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
75/75 [==============================] - 3s 5ms/step - loss: 6.8989 - accuracy: 0.0377
Epoch 2/50
75/75 [==============================] - 0s 5ms/step - loss: 6.6325 - accuracy: 0.0415
Epoch 3/50
75/75 [==============================] - 0s 5ms/step - loss: 6.3575 - accuracy: 0.0415
Epoch 4/50
75/75 

In [33]:
with open('data.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')
    texts = texts.replace('\xa0', '')


maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»„“', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])
dist = list(tokenizer.word_counts.items())


data = tokenizer.texts_to_sequences([texts])
sak = to_categorical(data[0], num_classes=maxWordsCount)
inp_words = 2
n = sak.shape[0] - inp_words
X = np.array([sak[i:i + inp_words, :] for i in range(n)])
Y = sak[inp_words:]


model = Sequential()
model.add(Input(shape=(inp_words, maxWordsCount)))
model.add(GRU(64, return_sequences=True))
model.add(GRU(32, return_sequences=True))
model.add(GRU(16, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=50)

sak = buildPhrase("taip", "daro")
print(sak)

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_10 (GRU)                (None, 2, 64)             204672    
                                                                 
 gru_11 (GRU)                (None, 2, 32)             9408      
                                                                 
 gru_12 (GRU)                (None, 16)                2400      
                                                                 
 dense_14 (Dense)            (None, 1000)              17000     
                                                                 
Total params: 233,480
Trainable params: 233,480
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
75/75 [==============================] - 5s 6ms/step - loss: 6.8730 - accuracy: 0.0381
Epoch 2/50
75/75 [==============================] - 0s 6ms/step - loss: 6.5230 - 

In [39]:
with open('data.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')
    texts = texts.replace('\xa0', '')


maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»„“', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])
dist = list(tokenizer.word_counts.items())


data = tokenizer.texts_to_sequences([texts])
sak = to_categorical(data[0], num_classes=maxWordsCount)
inp_words = 2
n = sak.shape[0] - inp_words
X = np.array([sak[i:i + inp_words, :] for i in range(n)])
Y = sak[inp_words:]


model = Sequential()
model.add(Input(shape=(inp_words, maxWordsCount)))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32,activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=70)

sak = buildPhrase("taip", "daro")
print(sak)

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_31 (SimpleRNN)   (None, 2, 32)             33056     
                                                                 
 simple_rnn_32 (SimpleRNN)   (None, 2, 32)             2080      
                                                                 
 simple_rnn_33 (SimpleRNN)   (None, 2, 32)             2080      
                                                                 
 simple_rnn_34 (SimpleRNN)   (None, 32)                2080      
                                                                 
 dense_20 (Dense)            (None, 1000)              33000     
                                                                 
Total params: 72,296
Trainable params: 72,296
Non-trainable params: 0
_________________________________________________________________
Epoch 1/70
75/75 [==============================]